<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 4: Practical - Naive Bayes <br> <br>
<font color=686880 size=4>
    TAs: Alireza Farashah - Arman Malekzadeh - Ali Salesi
    
____

### Full Name : Reza Soumi
### Student Number : 98105857
### Colab Link: https://colab.research.google.com/drive/1eMcHBqcQBSerZc7x6vAgBH3ry8SNwUDp?usp=sharing
___

Note 1: In this assignment, we are trying to simulate the functionality of something called "CountVectorizer" which is used in natural language processing. You are advised to take a look at this link before beginning to answer the questions:

https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

Note 2: One or two TA sessions will be held related to this assignment to make sure you get familiarized with this topic. Therefore, keep calm and write code!

# Download the Dataset

In [1]:
!wget -O /content/dataset.zip "https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1"

--2023-05-04 12:50:32--  https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/yf195wl0sp3term/dataset-train.zip [following]
--2023-05-04 12:50:32--  https://www.dropbox.com/s/dl/yf195wl0sp3term/dataset-train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbe183a47c2a5e118ebe9173332.dl.dropboxusercontent.com/cd/0/get/B7aOTyk8OqKJeypihsXcbjn2h-3f8S06XJPcFNSSAuxWCkUkEEmLRu5HWs30jueO30SeZqnRqPgcNfl-HAexcSSkmh7_9X3yinUEAQOhuLKIWhEDatqS9p5xKxD9dzv9CXPBa6tAK3rW7e_3-atWTWBcUGt95J4KKQlkZwDXMpFsKDV6g5pRHkT7F4a6ltxAJYc/file?dl=1# [following]
--2023-05-04 12:50:32--  https://ucbe183a47c2a5e118ebe9173332.dl.dropboxusercontent.com/cd/0/get/B7aOTyk8OqKJeypihsXcbjn2h-3f8S06XJPcFNSSAu

In [2]:
!mkdir /content/dataset/
!unzip '/content/dataset.zip' -d /content/dataset/

Archive:  /content/dataset.zip
  inflating: /content/dataset/content/news-train.csv  


# Preprocessing

In [27]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import resample
from collections import Counter

Read the csv file and load it as a dataframe.

In [10]:
df = pd.read_csv("dataset/content/news-train.csv")
df.describe()

,topic,text
count,197361,197361
unique,4,195932
top,sports,بازار در یک نگاه جدول شاخص‌های بورس
freq,123105,39


# Dealing with Class Imbalance

For each "topic", count and display the number of news belonging to it. (5 points)

In [12]:
topic_counts = df['topic'].value_counts()
print(topic_counts)

sports       123105
economics     49600
cultural      13359
politics      11297
Name: topic, dtype: int64


Balance the dataset in a way that all of the topics get associated with the same number of news. For this purpose, find the topic for which the minimum number of news exists. Then, use downsampling to lower the number of news associated with the other topics.<br>
Finally, save the result as a new dataframe. (15 points)

In [15]:
min_count = df['topic'].value_counts().min()
downsampled = pd.DataFrame()
for topic in df['topic'].unique():
    topic_df = df[df['topic'] == topic]
    downsampled_topic = resample(topic_df, replace=False, n_samples=min_count, random_state=42)
    downsampled = pd.concat([downsampled, downsampled_topic])

downsampled_counts = downsampled['topic'].value_counts()
print(downsampled_counts)

sports       11297
politics     11297
economics    11297
cultural     11297
Name: topic, dtype: int64


# Preparation and Feature Extraction

Split the dataframe to two parts for training (80%) and testing (20%). (5 points)

In [25]:
train_df, test_df = train_test_split(downsampled, test_size=0.2, random_state=42)

In this part, we will extract features from the textual data using a very basic method.

Consider the following sentence:
```
Sometimes a dog can run faster than a cat.
```
In the above sentence, the "tokens" are:

```
"sometimes", "a", "dog", "can", "run", "faster", "than", "a", "cat"
```

Find the tokens of each news article in the "training dataframe" by splitting it based on the occurence of the space character (5 points). For instance, the following sentence:
```
او به میدان رفت
```
gets converted to these tokens:
```
["او","به","میدان","رفت"]
```

In [26]:
train_df['text'] = train_df['text'].apply(lambda x: x.split(" "))
train_df

,topic,text
131394,politics,"[همه‌چیز, شبیه, مسابقه, هفته, بود, البته, بدون..."
88791,politics,"[رییس, مجمع, تشخیص, مصلحت, نظام, با, اشاره, به..."
121984,sports,"[مرحله, اول, مسابقات, لیگ, ورزش‌های, سه, گانه,..."
12673,politics,"[#, بارها, تاکید, کرده‌اند, #, برنامه‌ای, برای..."
106051,sports,"[به, گزارش, ورزش, سه, کمتر, کسی, فکر, می‌کرد, ..."
...,...,...
92968,sports,"[به, گزارش, ایسنا, جام‌جهانی, #, روسیه, #, روز..."
169808,cultural,"[نمایش, کمدی, #, به, کارگردانی, میثم, عبدی, از..."
157638,cultural,"[معاون, پژوهشی, جهاد, دانشگاهی, واحد, استان, ا..."
118852,sports,"[به, گزارش, ورزش, سه, آبی, پوشان, پایتخت, که, ..."


Count the tokens and list the top 5 frequent ones (5 points)

In [30]:
token_list = [word for row in train_df['text'] for word in row]

token_counts = Counter(token_list)

top_tokens = token_counts.most_common(5)

for token, count in top_tokens:
    print(f'{token}: {count}')

#: 1621876
و: 878306
در: 743093

: 589997
به: 580896


Consider the following sentence:
```
تیم فوتبال بارسلونا امشب با بایرن مونیخ دیدار می‌کند.
```
In the above sentence, we do not need the word `با` to determine that the topic is "sports". This word doesn't have any impact on the category this sentence belongs to. 
This word and similar ones are called "stopwords". A list of stopwords for the Persian language can be found in this text file:
https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

Use it to remove all tokens that are actually stopwords.
Also, remove the `#` token. (10 points)

In [31]:
!wget -O /content/stopwords.txt https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

--2023-05-04 13:36:58--  https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt [following]
--2023-05-04 13:36:58--  https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2942 (2.9K) [text/plain]
Saving to: ‘/content/stopwords.txt’

/content/stopwords. 100%[===================>]   2.87K  --.-KB/s    in 0s      

2023-05-04 13:36:58 (36.8 MB/s)

In [34]:
with open('stopwords.txt', 'r') as f:
    lines = f.readlines()
stopwords = [line.strip() for line in lines]
stopwords.append("#")

train_df['text'] = train_df['text'].apply(lambda x: [s for s in x if s not in stopwords])
train_df

,topic,text
131394,politics,"[همه‌چیز, شبیه, مسابقه, هفته, حضور, مجری, مرحو..."
88791,politics,"[رییس, مجمع, تشخیص, مصلحت, نظام, اشاره, ثبت‌نا..."
121984,sports,"[مرحله, مسابقات, لیگ, ورزش‌های, سه, گانه, برگز..."
12673,politics,"[بارها, تاکید, کرده‌اند, برنامه‌ای, برای, حذف,..."
106051,sports,"[گزارش, ورزش, سه, کمتر, فکر, می‌کرد, بازی, تیم..."
...,...,...
92968,sports,"[گزارش, ایسنا, جام‌جهانی, روسیه, روز, آغاز, می..."
169808,cultural,"[نمایش, کمدی, کارگردانی, میثم, عبدی, دی, سالن,..."
157638,cultural,"[معاون, پژوهشی, جهاد, دانشگاهی, واحد, استان, ا..."
118852,sports,"[گزارش, ورزش, سه, آبی, پوشان, پایتخت, نیم, فصل..."


Make a list of the remaining unique tokens (5 points)

In [35]:
unique_token_list = list(set([word for row in train_df['text'] for word in row]))

Count the number of the occurences of each token in each document. This way, you can make a numpy array to represent each document. Call this new array `x_train`. (10 points)

Example:

Unique tokens in all documents: 
```
0: hen
1: sometimes
2: a
3: the
4: dog
5: runs
6: faster
7: than
8: cat
9: bird
```

Current Document: `sometimes a dog runs faster than a cat`

The representation of the document:

```
[0, 1, 2, 0, 1, 1, 1, 1, 1, 0]
```

The meaning of this representation:

```
0: the document doesn't contain "hen"
1: the document contains 1 "sometimes"
2: the document contains 2 "a"
0: the document doesn't contain "the"
1: the document contains 1 "dog"
1: the document contains 1 "runs"
1: the document contains 1 "faster"
1: the document contains 1 "than"
1: the document contains 1 "cat"
0: the document doesn't contain "bird"
``` 

In [64]:
x_train = train_df['text'].to_numpy()
vectorizer = CountVectorizer(vocabulary=unique_token_list)
x_train = [vectorizer.fit_transform([' '.join(x)]).toarray().tolist()[0] for x in x_train]

Make another numpy array by converting the topics associated with the training news to numbers. For instance, if the topics are: `sports`, `economics`, `politics`, and `cultural`, convert them to `0` to `3`. Call this new array `y_train`. (5 points)

In [60]:
topic_dict = {
    "sports": 0,
    "economics": 1,
    "politics": 2,
    "cultural": 3
}

In [61]:
y_train = [topic_dict[t] for t in train_df['topic'].to_numpy()]
print(y_train)

[2, 2, 0, 2, 0, 1, 1, 3, 2, 2, 2, 0, 0, 3, 2, 2, 0, 3, 1, 0, 3, 0, 0, 0, 1, 0, 2, 3, 0, 3, 1, 3, 0, 1, 0, 1, 0, 2, 0, 1, 1, 3, 3, 1, 2, 2, 2, 2, 0, 3, 3, 1, 0, 1, 3, 0, 1, 0, 1, 0, 1, 3, 1, 3, 2, 2, 3, 0, 0, 1, 0, 1, 3, 1, 1, 1, 1, 2, 0, 1, 0, 1, 2, 0, 3, 3, 1, 3, 3, 3, 2, 1, 3, 3, 2, 0, 3, 2, 1, 1, 2, 2, 0, 2, 0, 0, 2, 2, 1, 3, 0, 1, 0, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 0, 3, 0, 2, 0, 1, 2, 3, 2, 1, 0, 2, 0, 1, 3, 2, 2, 1, 1, 1, 3, 2, 0, 3, 2, 0, 3, 3, 2, 2, 2, 0, 1, 0, 0, 2, 2, 2, 0, 1, 0, 2, 1, 1, 0, 3, 3, 0, 2, 0, 2, 0, 2, 1, 2, 3, 2, 3, 0, 2, 2, 0, 2, 3, 2, 1, 3, 3, 1, 2, 3, 2, 1, 2, 2, 0, 3, 1, 3, 0, 3, 3, 2, 0, 2, 2, 3, 3, 1, 3, 2, 2, 3, 0, 0, 0, 0, 1, 2, 0, 2, 3, 3, 0, 1, 1, 2, 2, 0, 1, 0, 3, 0, 0, 3, 0, 3, 0, 3, 3, 1, 3, 1, 0, 0, 3, 0, 2, 1, 0, 1, 3, 2, 1, 0, 2, 3, 2, 3, 0, 0, 0, 1, 0, 0, 2, 0, 2, 1, 1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 1, 2, 0, 2, 0, 0, 1, 1, 3, 3, 1, 0, 0, 0, 0, 0, 1, 3, 3, 3, 0, 1, 0, 0, 2, 0, 0, 1, 1, 0, 1, 1, 1, 1, 3, 1, 2, 1, 3, 0, 0, 2, 3, 1, 2, 3, 3, 1, 3, 2, 

Using the unique tokens you found in the training news, represent the testing news as another numpy array called `x_test`. (5 points)

Note: Do not forget to remove stopwords and the `#` token.

In [63]:
test_df['text'] = test_df['text'].apply(lambda x: x.split(" "))
test_df['text'] = test_df['text'].apply(lambda x: [s for s in x if s not in stopwords])
x_test = test_df['text'].to_numpy()
vectorizer = CountVectorizer(vocabulary=unique_token_list)
x_test = [vectorizer.fit_transform([' '.join(x)]).toarray().tolist()[0] for x in x_test]

Convert the topics associated with the testing news to their equivalent numbers (as before), and save the result as a numpy array called `y_test`. (5 points)

In [62]:
y_test = [topic_dict[t] for t in test_df['topic'].to_numpy()]

# Model Training and Evaluation

## Using sklearn (10 points)

Train a Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`) using `sklearn`.

In [65]:
clf = MultinomialNB()

clf.fit(x_train, y_train)

MultinomialNB()

Get the predictions of the model for the testing news (`x_test`).

In [66]:
y_test_pred = clf.predict(x_test)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [67]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      2253
           1       0.91      0.94      0.93      2268
           2       0.94      0.90      0.92      2262
           3       0.94      0.96      0.95      2255

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



## Using your own code (15 points)

In [78]:
print(len(x_train[0]))

4827


In [70]:
class Naive_Bayes:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        self.prior = None
        self.likelihood = None
    
    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        classes = np.unique(y)
        n_classes = len(classes)

        self.prior = np.zeros(n_classes)
        for i, c in enumerate(classes):
            self.prior[i] = np.sum(y == c) / n_samples

        self.likelihood = np.zeros((n_classes, n_features))
        for i, c in enumerate(classes):
            X_c = X[y == c]
            self.likelihood[i, :] = (np.sum(X_c, axis=0) + 1) / (np.sum(X_c) + n_features)

        return self
    
    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        Predicted value
        """
        
        n_samples, n_features = X.shape
        n_classes = len(self.prior)
        y_pred = np.zeros(n_samples)

        for i in range(n_samples):
            posterior = np.zeros(n_classes)
            for j in range(n_classes):
                posterior[j] = np.log(self.prior[j]) + np.sum(np.log(self.likelihood[j, :]) * X[i, :])
            y_pred[i] = np.argmax(posterior)

        return y_pred

Train your own Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`).

In [74]:
naive_bayes_model = Naive_Bayes()
naive_bayes_model.fit(np.array(x_train), np.array(y_train))

Get the predictions of your model for the testing news (`x_test`).

In [76]:
y_test_naive_bayes = naive_bayes_model.predict(np.array(x_test))

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [77]:
print(classification_report(y_test, y_test_naive_bayes))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      2253
           1       0.91      0.94      0.93      2268
           2       0.94      0.90      0.92      2262
           3       0.94      0.96      0.95      2255

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



# Kaggle

In [79]:
import csv
test_kaggle_df = pd.read_csv('assignment4-test-data.csv')
test_kaggle_df['text'] = test_kaggle_df['text'].apply(lambda x: x.split(" "))
test_kaggle_df['text'] = test_kaggle_df['text'].apply(lambda x: [s for s in x if s not in stopwords])
x_kaggle_test = test_kaggle_df['text'].to_numpy()
vectorizer = CountVectorizer(vocabulary=unique_token_list)
x_kaggle_test = [vectorizer.fit_transform([' '.join(x)]).toarray().tolist()[0] for x in x_kaggle_test]
y_test_kaggle_naive_bayes = naive_bayes_model.predict(np.array(x_kaggle_test))

with open('result.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for item in y_test_kaggle_naive_bayes:
        writer.writerow([item+1])

Competition Link: https://www.kaggle.com/t/88f1b6e251e34575b2e4cb4b91aed0ef